### Setup

In [80]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [81]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from irrep_bounds import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:

device = t.device("cuda" if t.cuda.is_available() else "cpu")
# MODEL_DIR = '2024-08-16_22-53-29_SL2_5_MLP3_256'
# MODEL_DIR = '2024-08-14_21-24-30_gapS_5_'
MODEL_DIR = '2024-09-17_17-05-38_gapS5_MLP3_128_wd2e-4_unembscale'
# MODEL_DIR = '2024-09-16_05-00-33_gapS5_MLP2_128_adamw_wd1'
# MODEL_DIR = '2024-09-13_03-46-47_gapS5_256_wd1e-3'
# MODEL_DIR = '2024-09-13_21-56-07_gapS5_MLP2_256_adamw_wd1'
# MODEL_DIR = '2024-08-27_16-32-46_Dic30_MLP3_128'
# MODEL_DIR = '2024-08-27_23-16-05_F11_MLP3_256'
# MODEL_DIR = '2024-09-02_18-17-20_smallgrp_54_8_MLP3_256_0_6'
# MODEL_DIR = '2024-09-05_15-22-58_smallgrp_54_8_MLP3_1024_0_6'
# MODEL_DIR = '2024-09-05_20-47-44_smallgrp_54_8_MLP3_1024_0_6_wd1e-6'
# MODEL_DIR = '2024-09-02_18-51-29_smallgrp_96_71_MLP3_256_0_6'
# MODEL_DIR = '2024-09-03_17-28-20_smallgrp_192_1025_MLP3_512_0_6'
# MODEL_DIR = '2024-09-03_21-09-07_Z_59_128'
disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir)
data = GroupData(params)
group = data.groups[0]

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: gapS(5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


/home/wilson/Finite-groups/src/model_utils.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = MODEL_DICT[params.model](params=params)


In [93]:
params

Parameters(instances=100, embed_dim=128, hidden_size=128, epochs=2000000, batch_size=64, batched=False, activation='relu', weight_decay=0.0002, lr=0.01, beta1=0.9, beta2=0.98, optimizer='adam', checkpoint=5000, name='gapS5_MLP3_128_wd2e-4_unembscale', seed=42, group_string='gapS(5)', intersect_frac=1.0, delta_frac=[0.0], train_frac=0.4, save_weights=True, save_losses=False, load_weights='', wandb=True, thresh_grok=0.95, project='group generalization', model='MLP3', unembed_bias=False, init_func='kaiming_uniform', correct_embed=False)

In [94]:
loss_dict = test_loss(models[0].to(device), data)

In [101]:

# instance = loss_dict['G0_acc'].argmax().item()
# instance = loss_dict['G0_loss'].argmin().item()
# instance = (loss_dict['G0_acc'] > 0.999).nonzero()[11]
instance = 1
print(loss_dict[f'G0_loss'][instance], loss_dict[f'G0_acc'][instance], instance)
model = models[0][instance].to(device)

tensor(0.0298) tensor(1.) 1


In [104]:
model = MLP3(params)

In [102]:
irreps, irrep_idx_dict = get_neuron_irreps(model, group)

In [103]:
vecs = get_neuron_vecs(model, group, irreps, irrep_idx_dict)

1d-0
1-r2 90th percentile 0.0
a variance: 0.0
b variance: 1.0
c variance: 1.0
d variance: 0.0
a vs d tensor(0.)
2 0.0
2 0.0
b has 2 clusters with total loss 0.0
c has 2 clusters with total loss 0.0
b_parts [[0, 1]]
c_parts [[0, 1]]
a_mean tensor([1.])
replacing c_0 with -b_0
replacing c_1 with -b_1
b_mean tensor([[-1.],
        [ 1.]])
c_mean tensor([[ 1.],
        [-1.]])
b_labels tensor([1, 0], dtype=torch.int32)
c_labels tensor([1, 0], dtype=torch.int32)
coef tensor([1.9251, 1.9256])
unif_coef tensor([1.9254, 1.9254])
coef diff tensor(1.7318e-08)
dict_keys(['1d-0'])

4d-0
5d-0
6d-0
5d-1
4d-1
1-r2 90th percentile 0.9945856332778931
a variance: 6.158730343486241e-07
b variance: 0.9110554456710815
c variance: 0.9125399589538574
d variance: 1.0327377140129101e-06
a vs d tensor(1.9587e-06)
2 6325.4423828125
3 5046.5546875
4 3958.37060546875
5 2907.12890625
6 2688.544189453125
7 2490.0634765625
8 2305.787353515625
9 2139.576416015625
10 1986.288818359375
11 1853.218017578125
12 1712.42114

KeyboardInterrupt: 

In [88]:
coef, A_norm, a_mean, b_mean, c_mean, b_labels, c_labels, b_parts, c_parts = vecs['4d-1']

In [89]:
ideal = get_idealized_model(model, irreps, irrep_idx_dict, vecs)

1d-0
l diff tensor(2.2452e-05)
r diff tensor(1.6083e-05)
u diff tensor(0.0005)
4d-1
l diff tensor(6.5694e-05)
r diff tensor(7.2110e-05)
u diff tensor(0.0365)
total
l diff tensor(6.2486e-05)
r diff tensor(6.7950e-05)
u diff tensor(0.0333)


In [90]:
model_dist(model, ideal)

l diff tensor(0.4933)
r diff tensor(0.5232)
u diff tensor(2.1147)
l norm tensor(37.6290)
r norm tensor(37.6470)
u norm tensor(8.9418)


tensor(168.0637)

In [91]:
d = model_dist_inf(model, ideal).item()
d

15.990200996398926

In [78]:
out = ideal(t.tensor([[0, 1]]))
((out.max() - out) > d).float().mean()

tensor(0.5250)

In [31]:
model = model.fold_linear()
# ideal = ideal.fold_linear()
ln1, rn1, un1 = model.get_neurons()
ln2, rn2, un2 = ideal.get_neurons()
ln1, rn1, un1 = ln1[:,:, part_idxs], rn1[:,:, part_idxs], un1[:,:, part_idxs]
ln2, rn2, un2 = ln2[:,:, part_idxs], rn2[:,:, part_idxs], un2[:,:, part_idxs]
ln_noise = t.randn_like(ln1)
ln_noise = ln_noise / ln_noise.norm() * (ln1 - ln2).norm()
rn_noise = t.randn_like(rn1)
rn_noise = rn_noise / rn_noise.norm() * (rn1 - rn2).norm()
un_noise = t.randn_like(un1)
un_noise = un_noise / un_noise.norm() * (un1 - un2).norm()
ln3, rn3, un3 = ln2 + ln_noise, rn2 + rn_noise, un2 + un_noise

NameError: name 'part_idxs' is not defined

In [15]:
un1.norm(), un2.norm() 

(tensor(27.0105), tensor(26.9070))

In [20]:
noise_model = copy.deepcopy(model)
noise_model.embedding_left = nn.Parameter(ln3)
noise_model.embedding_right = nn.Parameter(rn3)
noise_model.unembedding = nn.Parameter(un3.mT)

In [21]:
test_loss(noise_model, data)

{'G0_loss': tensor([0.2678]),
 'G0_loss_std': tensor([0.1247]),
 'G0_acc': tensor([1.])}

In [57]:
test_loss(noise_model, data), test_loss(ideal, data),  test_loss(model, data)

({'G0_loss': tensor([0.1731]),
  'G0_loss_std': tensor([0.1883]),
  'G0_acc': tensor([0.9990])},
 {'G0_loss': tensor([0.1231]),
  'G0_loss_std': tensor([0.0968]),
  'G0_acc': tensor([1.])},
 {'G0_loss': tensor([0.0296]),
  'G0_loss_std': tensor([0.0080]),
  'G0_acc': tensor([1.])})

In [29]:
model_dist2(model, ideal)#, irrep_idx_dict, vecs)

tensor(151.1256)

In [28]:
sorted(dist.tolist())

TypeError: 'float' object is not iterable

In [44]:
model_dist_parted(model, ideal, irrep_idx_dict, vecs)

1d-0
l diff tensor(0.1423)
r diff tensor(0.1301)
u diff tensor(0.1309)
l norm tensor(37.6832)
r norm tensor(37.6227)
u norm tensor(8.6710)
12.21975326538086
4d-1
l diff tensor(0.5003)
r diff tensor(0.4084)
u diff tensor(2.0580)
l norm tensor(37.3536)
r norm tensor(37.2156)
u norm tensor(6.8900)
159.40463256835938
4d-1
l diff tensor(0.)
r diff tensor(0.)
u diff tensor(1.4427)
l norm tensor(8.8416)
r norm tensor(8.7849)
u norm tensor(1.4427)
25.430316925048828


197.05470275878906

In [32]:
_, _, un1 = model.get_neurons(True)
_, _, un2 = ideal.get_neurons(True)

In [43]:
((un1 - un2).norm(dim=0))[part_idxs].max() #/ un1.norm(dim=0))[part_idxs]

tensor(0.5175)

In [61]:
(ideal.unembedding - model.unembedding)[0].norm(dim=1)

tensor([3.9519e-01, 1.9263e+00, 2.7491e-02, 0.0000e+00, 2.0834e-03, 2.9060e-02,
        3.0195e-01, 4.8089e-01, 1.3822e+00, 2.9299e-01, 5.6391e-04, 1.7642e+00,
        4.3101e-01, 2.3929e-01, 3.8706e-04, 1.8467e-01, 3.0984e-01, 6.9950e-03,
        1.4025e+00, 1.1464e-01, 0.0000e+00, 2.8585e-01, 1.7904e+00, 2.1838e-01,
        1.2053e-04, 4.7698e-01, 1.7771e+00, 4.6449e-02, 1.2915e-04, 1.2651e-01,
        4.9187e-03, 1.3275e-01, 1.5971e+00, 8.5666e-02, 4.7559e-01, 0.0000e+00,
        9.9433e-02, 3.8776e-01, 1.8291e+00, 8.8188e-01, 9.8395e-02, 1.2016e-04,
        0.0000e+00, 4.6339e-01, 3.0990e-02, 1.6035e-02, 4.7704e-01, 1.9921e-03,
        4.4426e-01, 3.2006e-01, 2.2652e-01, 1.8414e+00, 1.0796e+00, 1.7409e-02,
        8.8652e-02, 4.2277e-01, 1.8674e+00, 1.2950e-03, 4.2510e-01, 4.7889e-01,
        1.8257e+00, 8.6694e-02, 8.8028e-02, 4.6159e-01, 1.5211e+00, 1.6752e+00,
        1.5701e-02, 1.8629e+00, 4.4153e-01, 3.2458e-01, 1.3208e-03, 4.6803e-02,
        0.0000e+00, 1.5985e-03, 1.2449e+

In [43]:
ideal.unembedding[0, part_idxs[4], :15], model.unembedding[0, part_idxs[4], :15]

(tensor([-0.9858,  0.2464,  0.2464,  0.2464,  0.2464, -0.9858, -0.9858,  0.2464,
          0.2464,  0.2464,  0.2464, -0.9858,  0.2464,  0.2464,  0.2464],
        grad_fn=<SliceBackward0>),
 tensor([-0.9570,  0.1998,  0.2163,  0.2846,  0.2874, -1.0399, -1.0421,  0.2721,
          0.2724,  0.2598,  0.1809, -0.9766,  0.2939,  0.1614,  0.2129],
        grad_fn=<SliceBackward0>))

In [37]:
irrep_idx_dict['4d-1'].index(88)

65

In [38]:
b_labels[65], c_labels[65]

(tensor(5, dtype=torch.int32), tensor(5, dtype=torch.int32))

In [12]:
from coset_bounds import model_dist as cdist

In [15]:
cdist(model, ideal)

l diff tensor(0.5003)
r diff tensor(0.4084)
u diff tensor(2.0830)
l norm tensor(37.6832)
r norm tensor(37.6227)
u norm tensor(8.9174)


tensor(164.9650)

In [136]:
ideal.unembedding[0,5,:].norm(), model.unembedding[0,5,:].norm()

(tensor(0., grad_fn=<LinalgVectorNormBackward0>),
 tensor(0.1013, grad_fn=<LinalgVectorNormBackward0>))

In [148]:
(ideal.unembedding[0] - model.unembedding[0]).norm(dim=1).argmax()

tensor(88)

In [150]:
ideal.unembedding[0,88].norm(), model.unembedding[0,88].norm()

(tensor(0., grad_fn=<LinalgVectorNormBackward0>),
 tensor(1.2796, grad_fn=<LinalgVectorNormBackward0>))

In [154]:
test_loss(ideal, data), test_loss(model, data)

({'G0_loss': tensor([0.1231]),
  'G0_loss_std': tensor([0.0968]),
  'G0_acc': tensor([1.])},
 {'G0_loss': tensor([0.0296]),
  'G0_loss_std': tensor([0.0080]),
  'G0_acc': tensor([1.])})

In [163]:
ideal(t.tensor([[10, 15]])).max()

tensor(15.5970, grad_fn=<MaxBackward1>)

In [173]:
irrep_model = copy.deepcopy(ideal)
irrep_name = '1d-0'
irrep_model.embedding_left.data = ideal.embedding_left[:,:,irrep_idx_dict[irrep_name]]
irrep_model.embedding_right.data = ideal.embedding_right[:,:,irrep_idx_dict[irrep_name]]
irrep_model.unembedding.data = ideal.unembedding[:,irrep_idx_dict[irrep_name]]

In [171]:
test_loss(irrep_model, data)

{'G0_loss': tensor([0.2198]),
 'G0_loss_std': tensor([4.2085e-07]),
 'G0_acc': tensor([1.])}

In [180]:
irrep_model(t.tensor([[11, 17]]))

tensor([[[-3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460,  3.8460,
          -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460,
           3.8460, -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,
           3.8460,  3.8460, -3.8460,  3.8460, -3.8460, -3.8460,  3.8460,
           3.8460, -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,
           3.8460,  3.8460, -3.8460, -3.8460,  3.8460,  3.8460, -3.8460,
          -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460, -3.8460,
           3.8460,  3.8460, -3.8460, -3.8460,  3.8460,  3.8460, -3.8460,
          -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460,  3.8460,
          -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460,
           3.8460, -3.8460,  3.8460, -3.8460, -3.8460,  3.8460,  3.8460,
          -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,  3.8460,
           3.8460, -3.8460, -3.8460,  3.8460,  3.8460, -3.8460, -3.8460,
           3.8460,  3.8460, -3.8460, -3.8460,  3.84

In [183]:
irrep_model.unembedding[0,:,:10]

tensor([[-0.5594,  0.5594,  0.5594, -0.5594, -0.5594,  0.5594,  0.5594, -0.5594,
         -0.5594,  0.5594],
        [-0.5598,  0.5598,  0.5598, -0.5598, -0.5598,  0.5598,  0.5598, -0.5598,
         -0.5598,  0.5598],
        [-0.0010,  0.0010,  0.0010, -0.0010, -0.0010,  0.0010,  0.0010, -0.0010,
         -0.0010,  0.0010]], grad_fn=<SliceBackward0>)